# Fraudulent Activities - PART I


In [84]:
#import libraries
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import sys
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.tree import plot_tree
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import  OneHotEncoder
from sklearn.metrics import precision_score, recall_score, confusion_matrix

from scipy import stats

from IPython.display import display

In [85]:
#load data
df = pd.read_csv("s3://full-stack-bigdata-datasets/Machine Learning Supervisé/projects/fraudulent_activity/Fraud_Data.csv")
ip_country = pd.read_csv("s3://full-stack-bigdata-datasets/Machine Learning Supervisé/projects/fraudulent_activity/IpAddress_to_Country.csv")

In [44]:
#connect IP address with country
def transform_ip(arg):
    try:
        return ip_country.country[(ip_country.lower_bound_ip_address < arg) & (ip_country.upper_bound_ip_address > arg)].iloc[0]
    except IndexError:
        return "Unknown country"  
      
df["country_name"] = df.ip_address.apply(transform_ip)

In [45]:
print(df.shape)
print(df.head())

(151112, 12)
   user_id          signup_time        purchase_time  purchase_value  \
0    22058  2015-02-24 22:55:49  2015-04-18 02:47:11              34   
1   333320  2015-06-07 20:39:50  2015-06-08 01:38:54              16   
2     1359  2015-01-01 18:52:44  2015-01-01 18:52:45              15   
3   150084  2015-04-28 21:13:25  2015-05-04 13:54:50              44   
4   221365  2015-07-21 07:09:52  2015-09-09 18:40:53              39   

       device_id source browser sex  age    ip_address  class     country_name  
0  QVPSPJUOCKZAR    SEO  Chrome   M   39  7.327584e+08      0            Japan  
1  EOGFQPIZPYXFZ    Ads  Chrome   F   53  3.503114e+08      0    United States  
2  YSSKYOSJHPPLJ    SEO   Opera   M   53  2.621474e+09      1    United States  
3  ATGTXKYKUDUQN    SEO  Safari   M   41  3.840542e+09      0  Unknown country  
4  NAUITBZFJKHWW    Ads  Safari   M   45  4.155831e+08      0    United States  


In [46]:
#drop user id
df = df.iloc[:, 1:]
df.head()

,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class,country_name
0,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0,Japan
1,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0,United States
2,2015-01-01 18:52:44,2015-01-01 18:52:45,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,1,United States
3,2015-04-28 21:13:25,2015-05-04 13:54:50,44,ATGTXKYKUDUQN,SEO,Safari,M,41,3.840542e+09,0,Unknown country
4,2015-07-21 07:09:52,2015-09-09 18:40:53,39,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,0,United States


In [47]:
#deal with date and time
df.signup_time = pd.to_datetime(df.signup_time, format="%Y-%m-%d %H:%M:%S")
df.purchase_time = pd.to_datetime(df.purchase_time, format="%Y-%m-%d %H:%M:%S")
df["time_delta"] = (df.purchase_time - df.signup_time) / np.timedelta64(1, 's')

In [48]:
df['year'] = pd.DatetimeIndex(df['purchase_time']).year
df['month'] = pd.DatetimeIndex(df['purchase_time']).month

df.head()

,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class,country_name,time_delta,year,month
0,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0,Japan,4506682.0,2015,4
1,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0,United States,17944.0,2015,6
2,2015-01-01 18:52:44,2015-01-01 18:52:45,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,1,United States,1.0,2015,1
3,2015-04-28 21:13:25,2015-05-04 13:54:50,44,ATGTXKYKUDUQN,SEO,Safari,M,41,3.840542e+09,0,Unknown country,492085.0,2015,5
4,2015-07-21 07:09:52,2015-09-09 18:40:53,39,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,0,United States,4361461.0,2015,9


In [49]:
#rename the target variable "class" to y
df = df.rename(columns={"class": "y"})

In [50]:
print(df.shape)
df.head()

(151112, 14)


,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,y,country_name,time_delta,year,month
0,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0,Japan,4506682.0,2015,4
1,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0,United States,17944.0,2015,6
2,2015-01-01 18:52:44,2015-01-01 18:52:45,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,1,United States,1.0,2015,1
3,2015-04-28 21:13:25,2015-05-04 13:54:50,44,ATGTXKYKUDUQN,SEO,Safari,M,41,3.840542e+09,0,Unknown country,492085.0,2015,5
4,2015-07-21 07:09:52,2015-09-09 18:40:53,39,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,0,United States,4361461.0,2015,9


In [51]:
df.describe(include='all')

/Users/katie/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  """Entry point for launching an IPython kernel.
/Users/katie/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  """Entry point for launching an IPython kernel.


,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,y,country_name,time_delta,year,month
count,151112,151112,151112.000000,151112,151112,151112,151112,151112.000000,1.511120e+05,151112.000000,151112,1.511120e+05,151112.0,151112.000000
unique,151112,150679,NaN,137956,3,5,2,NaN,NaN,NaN,182,NaN,NaN,NaN
top,2015-08-12 16:44:20,2015-07-17 23:22:55,NaN,EQYVNEGOFLAWK,SEO,Chrome,M,NaN,NaN,NaN,United States,NaN,NaN,NaN
freq,1,3,NaN,20,60615,61432,88293,NaN,NaN,NaN,58049,NaN,NaN,NaN
first,2015-01-01 00:00:42,2015-01-01 00:00:44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,2015-08-18 04:40:29,2015-12-16 02:56:05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,36.935372,NaN,NaN,NaN,NaN,33.140704,2.152145e+09,0.093646,NaN,4.932029e+06,2015.0,6.008629
std,NaN,NaN,18.322762,NaN,NaN,NaN,NaN,8.617733,1.248497e+09,0.291336,NaN,3.126263e+06,0.0,2.660637
min,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,18.000000,5.209350e+04,0.000000,NaN,1.000000e+00,2015.0,1.000000
25%,NaN,NaN,22.000000,NaN,NaN,NaN,NaN,27.000000,1.085934e+09,0.000000,NaN,2.186754e+06,2015.0,4.000000


In [52]:
#drop columns with excessive number of unique values
df = df.drop(['signup_time', 'purchase_time', 'device_id'], axis=1)

In [53]:
#drop IP address since we processed it earlier
df = df.drop(['ip_address'], axis=1)

In [54]:
#drop year because it's a constant
df = df.drop(['year'], axis=1)

In [55]:
#check number of instances per country
df['country_name'].value_counts()

United States                     58049
Unknown country                   21966
China                             12038
Japan                              7306
United Kingdom                     4490
                                  ...  
Yemen                                 1
British Indian Ocean Territory        1
Tajikistan                            1
Cape Verde                            1
Saint Martin                          1
Name: country_name, Length: 182, dtype: int64

In [56]:
#list those with more than 50 instances
country_counts = df['country_name'].value_counts()
print(country_counts[country_counts > 50].index.values)

['United States' 'Unknown country' 'China' 'Japan' 'United Kingdom'
 'Korea Republic of' 'Germany' 'France' 'Canada' 'Brazil' 'Italy'
 'Australia' 'Netherlands' 'Russian Federation' 'India'
 'Taiwan; Republic of China (ROC)' 'Mexico' 'Sweden' 'Spain'
 'South Africa' 'Switzerland' 'Poland' 'Argentina' 'Indonesia' 'Norway'
 'Colombia' 'Turkey' 'Viet Nam' 'Romania' 'Denmark' 'Hong Kong' 'Finland'
 'Austria' 'Ukraine' 'Chile' 'Belgium' 'Iran (ISLAMIC Republic Of)'
 'Egypt' 'Czech Republic' 'Thailand' 'New Zealand' 'Israel' 'Saudi Arabia'
 'Venezuela' 'Ireland' 'European Union' 'Greece' 'Portugal' 'Hungary'
 'Malaysia' 'Singapore' 'Pakistan' 'Philippines' 'Bulgaria' 'Morocco'
 'Algeria' 'Peru' 'Tunisia' 'United Arab Emirates' 'Ecuador' 'Seychelles'
 'Lithuania' 'Kenya' 'Kazakhstan' 'Kuwait' 'Costa Rica' 'Slovenia'
 'Slovakia (SLOVAK Republic)' 'Uruguay' 'Croatia (LOCAL Name: Hrvatska)'
 'Luxembourg' 'Belarus' 'Serbia' 'Nigeria' 'Latvia' 'Panama' 'Bolivia'
 'Dominican Republic']


In [57]:
#drop countries with less than 50
to_keep = country_counts[country_counts > 50].index.values
df1 = df1.loc[df1['country_name'].isin(to_keep), :]

In [58]:
#separate independant and dependent variables
target_name = 'y'

y = df.loc[:,target_name]
x = df.loc[:,[c for c in df.columns if c!=target_name]] 

In [59]:
print(x.head())
print(y.head())

   purchase_value source browser sex  age     country_name  time_delta  month
0              34    SEO  Chrome   M   39            Japan   4506682.0      4
1              16    Ads  Chrome   F   53    United States     17944.0      6
2              15    SEO   Opera   M   53    United States         1.0      1
3              44    SEO  Safari   M   41  Unknown country    492085.0      5
4              39    Ads  Safari   M   45    United States   4361461.0      9
0    0
1    0
2    1
3    0
4    0
Name: y, dtype: int64


In [60]:
#convert these dataframes to arrays
x = x.to_numpy()
y = y.to_numpy()

In [61]:
#separate data into training and test set
x_train, x_test, y_train, y_test = train_test_split(x, 
                                                    y, 
                                                    test_size=0.2, 
                                                    random_state=0, 
                                                    stratify=y)

In [62]:
#create pipelines for numerical and categorical variables
numeric_features = [0, 4, 6, 7]
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_features = [1, 2, 3, 5]
categorical_transformer = Pipeline(
    steps=[
    ('encoder', OneHotEncoder(drop='first'))
    ])

In [63]:
#make preprocessing object
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [82]:
np.set_printoptions(threshold=sys.maxsize)

In [83]:
print(x_test)

[[69 'Ads' 'IE' 'M' 32 'United States' 5792422.0 10]
 [35 'SEO' 'IE' 'M' 31 'China' 3808317.0 9]
 [16 'Ads' 'IE' 'M' 35 'United States' 5251566.0 5]
 [42 'Ads' 'Chrome' 'M' 29 'Iceland' 8213433.0 8]
 [30 'SEO' 'Opera' 'F' 35 'Japan' 252137.0 6]
 [21 'SEO' 'Safari' 'M' 27 'Unknown country' 8492295.0 6]
 [30 'Direct' 'Safari' 'M' 38 'Ireland' 506734.0 4]
 [48 'Ads' 'Chrome' 'F' 18 'China' 6526915.0 9]
 [30 'SEO' 'IE' 'M' 38 'United States' 2235085.0 3]
 [25 'Direct' 'Chrome' 'F' 39 'China' 2444431.0 9]
 [32 'Ads' 'Opera' 'M' 27 'Italy' 8896022.0 11]
 [37 'Direct' 'Safari' 'M' 22 'United States' 1.0 1]
 [50 'Ads' 'IE' 'F' 23 'United States' 10227378.0 11]
 [66 'Ads' 'Chrome' 'M' 36 'United States' 7876872.0 5]
 [77 'Ads' 'IE' 'F' 42 'Unknown country' 4616536.0 4]
 [44 'Ads' 'FireFox' 'M' 29 'Unknown country' 7513360.0 5]
 [52 'Ads' 'FireFox' 'F' 29 'United States' 7131891.0 8]
 [29 'SEO' 'FireFox' 'F' 25 'United States' 8142221.0 7]
 [86 'SEO' 'Chrome' 'F' 18 'China' 1.0 1]
 [35 'SEO' 'Fi

In [81]:
np.set_printoptions(threshold=False)

In [64]:
#preprocess on train and test sets
x_train = preprocessor.fit_transform(x_train)
x_test = preprocessor.transform(x_test)

ValueError: Found unknown categories ['San Marino', 'Antigua and Barbuda', 'Saint Martin', 'Bhutan', 'Myanmar'] in column 3 during transform

In [ ]:
#train baseline model
model = LogisticRegression(class_weight = 'balanced', max_iter = 1000)
model.fit(x_train, y_train)

#make predictions on training set
y_train_pred = model.predict(x_train)

#make predictions on test set
y_test_pred = model.predict(x_test)

#f1 scores
print("f1-score on training set : ", f1_score(y_train, y_train_pred))
print("f1-score on test set : ", f1_score(y_test, y_test_pred))

In [ ]:
#train random forest with grid search
rf = RandomForestClassifier()

params = {
    'n_estimators': [100, 120, 140, 160, 180],
    'max_depth': [4, 6, 8]
}

model = GridSearchCV(rf, param_grid = params, verbose=2)

model.fit(x_train, y_train)

print("Best hyperparameters : ")
print(model.best_params_)

In [ ]:
#make predictions on training set
y_train_pred = model.predict(x_train)
print(Y_train_pred[0:5])

#make predictions on test set
y_test_pred = model.predict(x_test)
print(y_test_pred[0:5])

# Print scores
print("f1-score on training set : ", f1_score(y_train, y_train_pred))
print("f1-score on test set : ", f1_score(y_test, y_test_pred))

In [78]:
print('--- Train set ---')
print('Precision : ', precision_score(y_train, y_train_pred))
print('Recall : ', recall_score(y_train, y_train_pred))
print()
print(confusion_matrix(y_train, y_train_pred))
print()
print('--- Test set ---')
print('Precision : ', precision_score(y_test, y_test_pred))
print('Recall : ', recall_score(y_test, y_test_pred))
print()
print(confusion_matrix(y_test, y_test_pred))
print()

--- Train set ---


NameError: name 'y_train_pred' is not defined